In [ ]:
!pip install pyspark
!pip install openclean

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 45.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=dd1eb9495c5ade99ca20cbee27062a7bde5c2fec90baf8ea444f72d31117840c
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
     |████████████████████████████████| 267 kB 4.4 MB/s 
     |████████████████████████████████| 109 kB 53.7 MB/s 
     |████████████████████████████████| 137 kB 48.5 MB/s 
     |████████████████████████████████| 260 kB 40.0 MB/s 
     |████████████████████████████████| 69 kB 6.1 MB/s 
     |████████████████████████████████| 525 kB 39.0 MB/s 
     |████████████████████████████████| 206 kB 33.5 MB/s 
     |████████████████████████████████| 596 kB 40.4 MB/s 
     |████████████████████████████████| 180 kB 55.9 MB/s 
     |████████████████████████████████| 149 kB 46.9 MB/s 
     |███████

# New Section

In [ ]:
#importing packages required
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import os
import requests
from six.moves import urllib
import sys 
import pandas as pd
import matplotlib 
import matplotlib as plt
import numpy as np
import scipy as sp
import IPython
from IPython import display
import sklearn
import random
import time
import warnings
import re
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Downloading file from NYC Open Data
fn_src = 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD'
fn_dst = 'NYPD_Complaint_Data_Historic.csv'

from six.moves import urllib

if os.path.isfile(fn_dst):
    print('File %s has already been downloaded' % fn_dst)
else:
    print('Fetching file %s[2.4GB]. This may take a while...' % fn_dst)
    urllib.request.urlretrieve(fn_src, fn_dst)
    print('File %s has been downloaded' % fn_dst)

Fetching file NYPD_Complaint_Data_Historic.csv[2.4GB]. This may take a while...
File NYPD_Complaint_Data_Historic.csv has been downloaded


In [ ]:
#Creating Spark Session
sc = SparkContext.getOrCreate();
spark = SparkSession(sc)

In [ ]:
#Reading the csv file
df_spark=spark.read.option("header",True).csv("/content/NYPD_Complaint_Data_Historic.csv",inferSchema=True) # data set 1.

## Data Profiling

In [ ]:
#Using openclean for finding anomalies
from openclean.pipeline import stream
ds = stream(fn_dst)

In [ ]:
#Creating profile of our dataset
from openclean.profiling.column import DefaultColumnProfiler
profiles = ds.profile(default_profiler=DefaultColumnProfiler)

In [ ]:
#CMPLNT_NUM, RPT_DT, KY_CD, LAW_CAT_CD have no null values
profiles.stats()

,total,empty,distinct,uniqueness,entropy
CMPLNT_NUM,7375993,0,7373143,9.996136e-01,22.813633
CMPLNT_FR_DT,7375993,655,8606,1.166862e-03,12.425578
CMPLNT_FR_TM,7375993,48,1441,1.953648e-04,8.136466
CMPLNT_TO_DT,7375993,1704204,6825,1.203324e-03,12.417984
CMPLNT_TO_TM,7375993,1699541,1441,2.538558e-04,8.862856
ADDR_PCT_CD,7375993,2166,77,1.044234e-05,6.148690
RPT_DT,7375993,0,5479,7.428152e-04,12.405384
KY_CD,7375993,0,74,1.003255e-05,4.170727
OFNS_DESC,7375993,18823,71,9.650450e-06,4.006583
PD_CD,7375993,6278,432,5.861828e-05,5.913459


In [ ]:
# Print the most frequent values in column 'OFNS_DESC'

profiles.column('OFNS_DESC').get('topValues')

[('PETIT LARCENY', 1244155),
 ('HARRASSMENT 2', 945389),
 ('ASSAULT 3 & RELATED OFFENSES', 774177),
 ('CRIMINAL MISCHIEF & RELATED OF', 744951),
 ('GRAND LARCENY', 638972),
 ('DANGEROUS DRUGS', 427058),
 ('OFF. AGNST PUB ORD SENSBLTY &', 383332),
 ('FELONY ASSAULT', 286085),
 ('ROBBERY', 266447),
 ('BURGLARY', 254292)]

In [ ]:
#Finding outliers i complaint dates
from openclean.profiling.anomalies.sklearn import DBSCANOutliers

incident_dates = ds.distinct('CMPLNT_FR_DT')
DBSCANOutliers().find(incident_dates)

['']

In [ ]:
incident_end_dates = ds.distinct('CMPLNT_TO_DT')
DBSCANOutliers().find(incident_end_dates)

['']

In [ ]:
#Finding outliers 
DBSCANOutliers(eps=0.05).find(incident_dates)

['',
 '08/25/1973',
 '01/01/2005',
 '12/31/2005',
 '05/26/1983',
 '02/22/2020',
 '01/01/2011',
 '12/30/2005',
 '01/01/2012',
 '03/14/2017',
 '02/02/2020',
 '12/27/2010',
 '01/01/2010',
 '03/28/1967',
 '02/20/2020']

In [ ]:
DBSCANOutliers(eps=0.05).find(incident_end_dates)

['', '03/28/1967']

## Checking if there are redundant rows. If present, remove them.

In [ ]:
# checking for Duplicate rows in our dataset.
df_spark.count() - df_spark.distinct().count()

0

In [ ]:
df_spark.printSchema()

root
 |-- CMPLNT_NUM: integer (nullable = true)
 |-- CMPLNT_FR_DT: string (nullable = true)
 |-- CMPLNT_FR_TM: string (nullable = true)
 |-- CMPLNT_TO_DT: string (nullable = true)
 |-- CMPLNT_TO_TM: string (nullable = true)
 |-- ADDR_PCT_CD: integer (nullable = true)
 |-- RPT_DT: string (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- PD_CD: integer (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- CRM_ATPT_CPTD_CD: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- BORO_NM: string (nullable = true)
 |-- LOC_OF_OCCUR_DESC: string (nullable = true)
 |-- PREM_TYP_DESC: string (nullable = true)
 |-- JURIS_DESC: string (nullable = true)
 |-- JURISDICTION_CODE: integer (nullable = true)
 |-- PARKS_NM: string (nullable = true)
 |-- HADEVELOPT: string (nullable = true)
 |-- HOUSING_PSA: string (nullable = true)
 |-- X_COORD_CD: integer (nullable = true)
 |-- Y_COORD_CD: integer (nullable = true)
 |-- SUSP_AG

## Removing the redundant columns

In [ ]:
#Check if there are redundant columns for latitude and longitude
print("Lat Lon column samples: ")
print(df_spark.select("Lat_Lon").show(10))
print("Latitude column samples: ")
print(df_spark.select("Latitude").show(10))
print("Longitude column samples: ")
print(df_spark.select("Longitude").show(10))

Lat Lon column samples: 
+--------------------+
|             Lat_Lon|
+--------------------+
|(40.8209267970000...|
|(40.8857014060000...|
|(40.7422811560000...|
|(40.8753114510000...|
|(40.698827283, -7...|
|(40.7207588210000...|
|(40.6959583620000...|
|(40.6115700660000...|
|(40.8402341380000...|
|(40.7728745600000...|
+--------------------+
only showing top 10 rows

None
Latitude column samples: 
+------------------+
|          Latitude|
+------------------+
| 40.82092679700002|
|40.885701406000074|
| 40.74228115600005|
| 40.87531145100007|
|      40.698827283|
| 40.72075882100006|
|40.695958362000056|
| 40.61157006600007|
| 40.84023413800003|
| 40.77287456000005|
+------------------+
only showing top 10 rows

None
Longitude column samples: 
+------------------+
|         Longitude|
+------------------+
|-73.94332421899996|
|-73.86164032499995|
|      -73.81982408|
|-73.84754521099995|
|     -73.938819047|
|-73.98046642299995|
|-73.93763162199998|
|-73.74736517199995|
|-73.91768411

We saw that the coordinates are redundant. So, we will remove the Latitude and Longitude columns

In [ ]:
df_spark=df_spark.drop("Latitude")
df_spark=df_spark.drop("Longitude")

# Cleaning the data if complaint number is invalid- 


In [ ]:
#Checking if the complaint number is valid
df2 = df_spark.filter(df_spark.CMPLNT_NUM.isNull()).drop()

In [ ]:
#We knew this from our profile analysis through openclean
df2.count()

0

In [ ]:
df_spark = df_spark.subtract(df2)

# Cleaning the data if jurisdiction code is invalid- 


In [ ]:
#Checking the validity of Jurisdiction code of the offense registered
df2 = df_spark.filter(df_spark.JURISDICTION_CODE.isNull()).drop()

In [ ]:
df2.count()

6278

In [ ]:
#Filtering invalid jurisdiction code
df_spark = df_spark.subtract(df2)

# Cleaning the data if precinct is invalid- 


In [ ]:
#Checking if it belongs to valid precinct
df3 = df_spark.filter(df_spark.ADDR_PCT_CD.isNull()).drop()

In [ ]:
df3.count()

384

In [ ]:
df_spark = df_spark.subtract(df3)

# Cleaning the columns associated with the dates- 
1. CMPLNT_FR_DT
2. CMPLNT_TO_DT
3. RPT_DT

In [ ]:
df_temp=df_spark.rdd

In [ ]:
#Deleting invalid dates
import re
import datetime
def valid_date_check(date):
  if date==None or date==" " or date=="":
      return False
  else:
    date_cpy=date
    date=date.split("/")
    try:
      month=int(date[0])
      day= int(date[1])
      year=int(date[2])
      if year>=2006 and year<=2020:
        try:
          refined_date=datetime.datetime(year, month, day)
          return True
        except:
          return False
      else:
        return False
    except:
      return False

In [ ]:
#filter the dates with proper format for Column-2 (CMPLNT_FR_DT)
df_temp=df_temp.map(lambda x:(x, valid_date_check(x[1]))).filter(lambda x: x[1]==True)
temp=df_temp.map(lambda x: x[0])

In [ ]:
#filter the dates with proper format for Column-4 (CMPLNT_TO_DT)
date_filtered_df=temp.map(lambda x:(x, valid_date_check(x[3]))).filter(lambda x: x[1]==True)
temp=date_filtered_df.map(lambda x: x[0])

In [ ]:
temp.count()

5656479

In [ ]:
temp=date_filtered_df.map(lambda x: x[0])
temp.take(10)

[Row(CMPLNT_NUM=413124026, CMPLNT_FR_DT='10/27/2019', CMPLNT_FR_TM='21:24:00', CMPLNT_TO_DT='10/27/2019', CMPLNT_TO_TM='21:24:00', ADDR_PCT_CD=20, RPT_DT='10/27/2019', KY_CD=351, OFNS_DESC='CRIMINAL MISCHIEF & RELATED OF', PD_CD=259, PD_DESC='CRIMINAL MISCHIEF,UNCLASSIFIED 4', CRM_ATPT_CPTD_CD='COMPLETED', LAW_CAT_CD='MISDEMEANOR', BORO_NM='MANHATTAN', LOC_OF_OCCUR_DESC='INSIDE', PREM_TYP_DESC='RESIDENCE - PUBLIC HOUSING', JURIS_DESC='N.Y. HOUSING POLICE', JURISDICTION_CODE=2, PARKS_NM=None, HADEVELOPT=None, HOUSING_PSA='45504.0', X_COORD_CD=988046, Y_COORD_CD=221545, SUSP_AGE_GROUP='25-44', SUSP_RACE='BLACK', SUSP_SEX='M', TRANSIT_DISTRICT=None, Lat_Lon='(40.774773578000065, -73.98629598399998)', PATROL_BORO='PATROL BORO MAN NORTH', STATION_NAME=None, VIC_AGE_GROUP='25-44', VIC_RACE='WHITE HISPANIC', VIC_SEX='F'),
 Row(CMPLNT_NUM=629183008, CMPLNT_FR_DT='10/04/2019', CMPLNT_FR_TM='21:00:00', CMPLNT_TO_DT='10/04/2019', CMPLNT_TO_TM='23:00:00', ADDR_PCT_CD=1, RPT_DT='10/06/2019', KY_CD=

## Cleaning the columns associated with the Time-

1. CMPLNT_FR_TM
2. CMPLNT_TO_TM

In [ ]:
#Deleting invalid time
def valid_time_check(time):
  if time==None or time==" " or time=="":
    return False
  else :
    cpy_time=time
    time=time.split(":")
    try:
      hour=int(time[0])
      mins=int(time[1])
      secs= int(time[2])
      # if hours is 24 then change it to 0 hours
      if hour == 24 and mins== 0 and secs == 0:
        hour=0
      try:
        newTime= datetime.time(hour,mins,secs)
        return True
      except :
        return False
    except:
      return False

In [ ]:
#filter the time with proper format for Column-3 (CMPLNT_FR_TM)
fr_time_filtered_df=temp.map(lambda x:(x, valid_time_check(x[2]))).filter(lambda x: x[1]==True)
temp=fr_time_filtered_df.map(lambda x: x[0])

In [ ]:
#filter the dates with proper format for Column-5 (CMPLNT_TO_TM)
to_time_filtered_df=temp.map(lambda x:(x, valid_time_check(x[4]))).filter(lambda x: x[1]==True)
temp=to_time_filtered_df.map(lambda x: x[0])

In [ ]:
temp.count()

5655465

# Cleaning the columns associated with the Level of offence-
1. LAW_CAT_CD

In [ ]:
#offense should belong to one of the categories- 'FELONY',"MISDEMEANOR","VIOLATION"
def level_of_offence_check(offence):
    if offence=="" or offence==" " or offence==None:
        return False
    else:
        crimes_list=['FELONY',"MISDEMEANOR","VIOLATION"]
        if offence not in crimes_list:
            return False
        else :
            return True

In [ ]:
#Filtering offence
offense_filtered_df=temp.map(lambda x:(x, level_of_offence_check(x[12]))).filter(lambda x: x[1]==True)
temp=offense_filtered_df.map(lambda x: x[0])


# Cleaning the columns associated with the valid borough name-
1. BORO_NM

In [ ]:
#Complaint should be registered in a valid NYC borough
def valid_borough_check(borough):
    if borough==None or borough=="" or borough==" ":
        return False
    else:
        borough_names=["MANHATTAN",'BRONX',"BROOKLYN","QUEENS","STATEN ISLAND"]
        if borough not in borough_names:
            return False
        else :
            return True

In [ ]:
borough_filtered_df=temp.map(lambda x:(x, valid_borough_check(x[13]))).filter(lambda x: x[1]==True)
temp=borough_filtered_df.map(lambda x: x[0])

# Cleaning the columns associated with the valid location of occurance-

1. LOC_OF_OCC_DEC

In [ ]:
#Offence should have occured at some place
def valid_location_check(loc):
    if loc is "" or loc is " ":
        return False
    else:
        loc_list=['FRONT OF','OPPOSITE OF','REAR OF','INSIDE','OUTSIDE']
        if loc not in loc_list:
            return False
        else :
            return True

In [ ]:
location_filtered_df=temp.map(lambda x:(x, valid_location_check(x[14]))).filter(lambda x: x[1]==True)
temp=location_filtered_df.map(lambda x: x[0])

# Cleaning the columns associated with the valid crimes(attempted or completed)-

1. CRM_ATPT_CPTD_CD

In [ ]:
#Crime should either be attempted or completed
def cmplt_attmpt_check(attmpt):
    if attmpt is "" or attmpt is " ":
        return False
    else:
        attmpt_code_list=['COMPLETED',"ATTEMPTED"]
        if attmpt not in attmpt_code_list:
            return False
        else:
            return True

In [ ]:
crime_filtered_df=temp.map(lambda x:(x, cmplt_attmpt_check(x[11]))).filter(lambda x: x[1]==True)
temp=crime_filtered_df.map(lambda x: x[0])

# Cleaning the columns associated with the valid Latitude and Longitude-

1. Lat_Lon

In [ ]:
#Checking the validity of coordinates
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
#Bounding Box LatLong details for NYC
lat_max=40.917577
lat_min=40.477399

long_max=-73.700009
long_min=-74.25909
#Function to check if lat long values present in NYC's Bounding Box
def is_present_nyc(point):
    if(point.x<lat_min or point.x>lat_max):
      return False
    if(point.y<long_min or point.y> long_max):
      return False
    return True

#Function to check validity of Lat Long values x[23]
def to_check_lat_long(x):
    if x== None or x== " "or x=="\t":
        return False
    x=x.strip("'")
    x=x.replace("(","")
    x=x.replace(")","")
    lat,lon=x.split(",")
    lat=lat.strip()
    lon=lon.strip()
    try:
        lat=float(lat)
        lon=float(lon)
        if is_present_nyc(Point(lat,lon)) :
            return True
        else:
            return False
    except:
        return False

In [ ]:
#Filtering the latitude and longitude that is invalid or is outside NYC
latlong_filtered_df=temp.map(lambda x:(x, to_check_lat_long(x[27]))).filter(lambda x: x[1]==True)
temp=latlong_filtered_df.map(lambda x: x[0])

# Cleaning the columns associated with suspect and victim's age group-

1. SUSP_AGE_GROUP
2. VIC_AGE_GROUP

In [ ]:
#Checking if age is not negative or greater than 100
def to_check_age(x):
    if x != None and x != " ":
        try:
          x=x.split("-")[0]
          age = int(x)
          if age<0 or age>100:
            return False
          else:
            return True
        except:
          return True
    return True

In [ ]:
#Filtering invalid age of suspect
suspect_age_filtered_df=temp.map(lambda x:(x, to_check_age(x[23]))).filter(lambda x: x[1]==True)
temp=suspect_age_filtered_df.map(lambda x: x[0])

In [ ]:
#Filtering invalid age of victim
victim_age_filtered_df=temp.map(lambda x:(x, to_check_age(x[30]))).filter(lambda x: x[1]==True)
temp=victim_age_filtered_df.map(lambda x: x[0])

In [ ]:
#Count of filtered or cleaned data available
temp.count()

4496605